In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Загружаем датасет

In [3]:
import json

data_file = "/content/drive/MyDrive/TelegramData/clean_dataset.json"


with open(data_file, 'r') as f:
  data = json.load(f)

len(data), type(data)

(4739, dict)

In [4]:
# зависимости
!pip install transformers sentence-transformers

import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset

In [5]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
device

device(type='cuda')

In [6]:
# model_embs = SentenceTransformer("cointegrated/rubert-tiny2")
# model_embs.cuda()


# all_comments = [c for com_list in comments
#                   for c in com_list]

# len(all_comments)

In [7]:
corpus = [(post, coms) for post in data.keys() for coms in data[post]]
len(corpus)

10516

In [8]:
corpus[1]

('Где заканчивается «Технофеодализм» Варуфакиса // Derrunda\n\nVK l Dzen l Web',
 'Вряд ли вернусь к нему Есть три альбома и хватит')

In [9]:
# кодируем в эмбеддинги комментарии посты
# comment_embs = model_embs.encode(all_comments, convert_to_numpy=True)
# post_embs = model_embs.encode(posts, convert_to_numpy=True)

# кластеризируем (получим кластеры на основе настроений комментариев.
# можно использовать отдельные модели для сентимент анализа и разбить комментарии прогоном через нее
# либо разбивать на кластеры и отдельно каждый кластер прогонять через сентимент модель.
# после чего для интерпретируемости называть классы по самым популярным настроениям
# так получим более адаптированную группировку по кластерам + интерпретируемость

# K = 5
# kmeans = KMeans(n_clusters=K, random_state=42)
# cluster_ids = kmeans.fit_predict(comment_embs)

# cur_idx = 0

# ft_dataset = []
# for post_idx, pairs in enumerate(corpus):
#   post = pairs[0]
#   comments_pair = pairs[1]
#   for comment in comments_pair:
#     ft_dataset.append([post, cluster_ids[cur_idx], comment])
#     cur_idx += 1

examples = [
    {
        # "input_text": f"Пост: {post} Тип реакции: {cluster}",
        "input_text": f"Пост: {post}",
        "output_text": comment
    }
    for post, comment in corpus
]

## 2. Препроцессинг

In [10]:
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
# определяем токен окончания генерации
if tokenizer.eos_token is None:
    tokenizer.eos_token = "</s>"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# препроцесс
MAX_POST_LEN = 128
MAX_TOTAL_LEN = 192

def preprocess(ex):
    prompt = ex["input_text"] + "\nКомментарий: "
    target = ex["output_text"] + tokenizer.eos_token

    prompt_ids = tokenizer(
        prompt,
        add_special_tokens=False,
        truncation=True,
        max_length=MAX_POST_LEN,
    )["input_ids"]

    target_ids = tokenizer(
        target,
        add_special_tokens=False,
        truncation=False,
    )["input_ids"]

    input_ids = prompt_ids + target_ids
    labels = [-100] * len(prompt_ids) + target_ids

    # финальное ограничение длины
    input_ids = input_ids[:MAX_TOTAL_LEN]
    labels = labels[:MAX_TOTAL_LEN]

    # ручной паддинг
    pad_len = MAX_TOTAL_LEN - len(input_ids)
    if pad_len > 0:
        input_ids += [tokenizer.pad_token_id] * pad_len
        labels += [-100] * pad_len

    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": [1] * (MAX_TOTAL_LEN - pad_len) + [0] * pad_len
    }
# добавление спецтокенов для кластеров
# special_tokens = {"additional_special_tokens": [f"<cluster_{i}>" for i in range(K)]}
# num_added = tokenizer.add_special_tokens(special_tokens)
# model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/149 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie transformer.wte.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
GPT2LMHeadModel LOAD REPORT from: sberbank-ai/rugpt3small_based_on_gpt2
Key                                     | Status     |  | 
----------------------------------------+------------+--+-
transformer.h.{0...11}.attn.masked_bias | UNEXPECTED |  | 
transformer.h.{0...11}.attn.bias        | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [11]:
dataset = Dataset.from_list(examples)
dataset = dataset.map(preprocess)

# разбиваем выборку
train_test = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test["train"]
val_test = train_test["test"].train_test_split(test_size=0.5, seed=42)
val_dataset, test_dataset = val_test["train"], val_test["test"]
# задаем параметры обучения модели
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    # gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,
    num_train_epochs=7,
    weight_decay=0.001,
    lr_scheduler_type="cosine",
    logging_dir="./logs",
    logging_steps=10,
    push_to_hub=False,
    fp16=torch.cuda.is_available(),
    report_to=["tensorboard"],
)

# создаем хф-трейнер
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

Map:   0%|          | 0/10516 [00:00<?, ? examples/s]

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [12]:
# запуск обучение
trainer.train()

# сохранение модели
save_dir = "/content/drive/MyDrive/diploma/weights_model"

trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,4.278204,4.161472
2,3.880581,4.171805
3,3.750317,4.234088
4,3.334383,4.313883
5,3.087455,4.397645
6,3.085976,4.438861
7,2.912572,4.456933


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('/content/drive/MyDrive/diploma/weights_model/tokenizer_config.json',
 '/content/drive/MyDrive/diploma/weights_model/tokenizer.json')

In [ ]:
input()

## Проверим результат на примере

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM

load_dir = "/content/drive/MyDrive/diploma/weights_model"

tokenizer = AutoTokenizer.from_pretrained(load_dir)
model = AutoModelForCausalLM.from_pretrained(load_dir).eval()

Loading weights:   0%|          | 0/149 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie transformer.wte.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [14]:
post_text = """В октябре световой день в Москве сократится на 2 часа 15 минут.

По данным Московского планетария, в начале месяца солнце будет светить 11 часов 34 минуты, а к концу — только 9 часов 19 минут."""
post_text = test_dataset["input_text"][225]
# cluster_id = 1  # тип реакции
prompt = f"{post_text} \nКомментарий:"

In [15]:
test_dataset["input_text"][225]

'Пост: 🤔 В Петербурге мужчина требует через суд вернуть 25 тысяч рублей, которые он переводил девушке «на такси».\n\nРоман и Елизавета познакомились в Telegram-канале в 2024 году, когда она ещё училась в школе. Мужчина переживал за её поездки и регулярно оплачивал такси.\n\nНо после того как мать девушки узнала о его банкротстве и судимостях, общение прекратилось. Теперь Роман требует вернуть деньги через суд.'

In [16]:
test_dataset["output_text"][225]

'Он же добровольно это делал.'

In [20]:
# если загружать веса модели без обучения, то инпут должен быть на цпу, если сразу после обучения, то нужно перекинуть на cuda
inputs = tokenizer(prompt, return_tensors="pt")#.to(device)

outputs = model.generate(
    **inputs,
    max_new_tokens=210,
    top_p=0.9,
    no_repeat_ngram_size=3,
    repetition_penalty=1.2,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
print(generated_text)

Пост: 🤔 В Петербурге мужчина требует через суд вернуть 25 тысяч рублей, которые он переводил девушке «на такси».

Роман и Елизавета познакомились в Telegram-канале в 2024 году, когда она ещё училась в школе. Мужчина переживал за её поездки и регулярно оплачивал такси.

Но после того как мать девушки узнала о его банкротстве и судимостях, общение прекратилось. Теперь Роман требует вернуть деньги через суд. 
Комментарий:А что если бы они просто так переписывались? И не было бы этого скандала?</s>


In [ ]:
# load_dir_1e = "/content/drive/MyDrive/diploma/weights_model_with_1_epoch"
save_dir_4e = "/content/drive/MyDrive/diploma/weights_model_with_4_epoch"

trainer.save_model(save_dir_4e)
tokenizer.save_pretrained(save_dir_4e)

('/content/drive/MyDrive/diploma/weights_model_with_1_epoch/tokenizer_config.json',
 '/content/drive/MyDrive/diploma/weights_model_with_1_epoch/special_tokens_map.json',
 '/content/drive/MyDrive/diploma/weights_model_with_1_epoch/vocab.json',
 '/content/drive/MyDrive/diploma/weights_model_with_1_epoch/merges.txt',
 '/content/drive/MyDrive/diploma/weights_model_with_1_epoch/added_tokens.json',
 '/content/drive/MyDrive/diploma/weights_model_with_1_epoch/tokenizer.json')

In [ ]:
sdir_1e = "/content/drive/MyDrive/diploma/weights_model_with_1_epoch"

checkpoint = "sberbank-ai/rugpt3small_based_on_gpt2"

model1 = AutoModelForCausalLM.from_pretrained(checkpoint)